# eval

In [ ]:
#!/usr/bin/env python3
import nlp
from datasets import load_dataset
from transformers import BertTokenizer, EncoderDecoderModel
tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16")
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16")
model.to("cuda")
test_dataset = nlp.load_dataset("cnn_dailymail", "3.0.0", split="test")
# test_dataset = load_dataset('cnn_dailymail', "3.0.0", split='test', ignore_verifications=True)

batch_size = 128
# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred"] = output_str
    return batch
results = test_dataset.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])
# load rouge for validation
rouge = nlp.load_metric("rouge")
pred_str = results["pred"]
label_str = results["highlights"]
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
print(rouge_output)

# error case analysis

In [1]:
from transformers import EncoderDecoderConfig, EncoderDecoderModel, AutoTokenizer

encoder_decoder_config = EncoderDecoderConfig.from_pretrained('./models/bert2bert_share/5/checkpoint-9000')
bert2bert_model = EncoderDecoderModel.from_pretrained('./models/bert2bert_share/5/checkpoint-9000', config=encoder_decoder_config)
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [2]:
import pandas as pd
from tweetNormalizer import normalizeTweet
# from nltk.tokenize import TweetTokenizer
# nltk_tknzr = TweetTokenizer()

# load train and validation data
train_dataset = pd.read_json("WNUT2015_dataset/train_data.json", orient="records")
val_dataset = pd.read_json("WNUT2015_dataset/test_truth.json", orient="records")

#make_sentence = lambda x : " ".join(x)
make_sentence = lambda x : normalizeTweet(" ".join(x))

train_dataset['input_sentence'] = train_dataset['input'].apply(make_sentence)
train_dataset['output_sentence'] = train_dataset['output'].apply(make_sentence)
val_dataset['input_sentence'] = val_dataset['input'].apply(make_sentence)
val_dataset['output_sentence'] = val_dataset['output'].apply(make_sentence)

In [3]:
tweet_text = "USERNAME WE cannot DOIN A FUCKEN MOVIE , it's so bad . lol. gonna to be better HTTPURL"
#tweet_text = train_dataset['input_sentence'].iloc[2]

input_ids = bert_tokenizer(tweet_text, return_tensors="pt").input_ids
output_ids = bert2bert_model.generate(input_ids)

print("tweet: ", tweet_text)
print("model: ", bert_tokenizer.convert_ids_to_tokens(output_ids[0], skip_special_tokens=True))
#print("model: ", " ".join(nltk_tknzr.tokenize(bert_tokenizer.decode(output_ids[0], skip_special_tokens=True))))

Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


tweet:  USERNAME WE cannot DOIN A FUCKEN MOVIE , it's so bad . lol. gonna to be better HTTPURL
model:  ['user', '##name', 'we', 'cannot', 'do', 'a', 'fucking', 'thing', ',', 'it', "'", 's', 'so', 'bad', '.', 'laughing', 'out', 'loud', '.', 'gonna', 'be', 'better', 'http', '##ur', '##l']
model:  username we cannot do a fucking thing , it's so bad . laughing out loud . gonna be better httpurl


In [4]:
tweet_text = "USERNAME WE cannot DOIN A FUCKEN MOVIE , it's so bad . lol. gonna to be better HTTPURL"
#tweet_text = train_dataset['input_sentence'].iloc[2]

input_ids = bert_tokenizer(tweet_text, return_tensors="pt").input_ids
output_ids = bert2bert_model.generate(input_ids)

print("tweet: ", tweet_text)
print("model: ", bert_tokenizer.decode(output_ids[0], skip_special_tokens=True))
#print("model: ", " ".join(nltk_tknzr.tokenize(bert_tokenizer.decode(output_ids[0], skip_special_tokens=True))))

Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


tweet:  USERNAME WE cannot DOIN A FUCKEN MOVIE , it's so bad . lol. gonna to be better HTTPURL
model:  username we cannot do a fucking thing , it's so bad . laughing out loud . gonna be better httpurl
model:  username we cannot do a fucking thing , it's so bad . laughing out loud . gonna be better httpurl


In [6]:
for i in range(0, 20):
#     print("example"+str(i))
    tweet_text = val_dataset.iloc[i]['input_sentence']
    normal_english = val_dataset.iloc[i]['output_sentence'].lower()

    input_ids = bert_tokenizer(tweet_text, return_tensors="pt").input_ids
    output_ids = bert2bert_model.generate(input_ids)

    print("tweet: ", tweet_text)
    print("model_orig: ", bert_tokenizer.decode(output_ids[0], skip_special_tokens=True))
    print("label: ", normal_english)
    print("model_nltk: ", " ".join(nltk_tknzr.tokenize(bert_tokenizer.decode(output_ids[0], skip_special_tokens=True))))
    print("label_nltk: ", " ".join(nltk_tknzr.tokenize(normal_english)))

Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


tweet:  username yeh but still that's wild lol
model_orig:  username yeah but still that's wild laughing out loud
label:  username yeah but still that's wild laughing out loud
model_nltk:  username yeah but still that's wild laughing out loud
label_nltk:  username yeah but still that's wild laughing out loud


Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


tweet:  Dick in Janice , Im poppin xanax and speakin spanish .
model_orig:  dick in janice , i'm poping xanax and speaking loud .
label:  dick in janice , i'm popping xanax and speaking spanish .
model_nltk:  dick in janice , i'm poping xanax and speaking loud .
label_nltk:  dick in janice , i'm popping xanax and speaking spanish .


Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


...

tweet:  username Ayy skiperooo wassup cuz what u mean by Marlton ?
model_orig:  username ayy skips was so what you know about what you mean ?
label:  username ayy skiperooo what's up because what you mean by marlton ?
model_nltk:  username ayy skips was so what you know about what you mean ?
label_nltk:  username ayy skiperooo what's up because what you mean by marlton ?


Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


tweet:  RT username : Sean , Darragh and I , in croke park ! Unbelievable weekend ! httpurl
model_orig:  rt username : sean , darragh and i , in croke park ! unbelievable weekend ! httpurl
label:  rt username : sean , darragh and i , in croke park ! unbelievable weekend ! httpurl
model_nltk:  rt username : sean , darragh and i , in croke park ! unbelievable weekend ! httpurl
label_nltk:  rt username : sean , darragh and i , in croke park ! unbelievable weekend ! httpurl


Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence


tweet:  username umm I thought that convo was pretty funny soooooo
model_orig:  username umm i thought that convo was pretty funny so damn
label:  username umm i thought that conversation was pretty funny so
model_nltk:  username umm i thought that convo was pretty funny so damn
label_nltk:  username umm i thought that conversation was pretty funny so
tweet:  Stop be so fucken gay you three aint nobody got time for that username username username
model_orig:  stop be so fucking gay you three people ain't nobody got that username username
label:  stop be so fucking gay you three ain't nobody got time for that username username username
model_nltk:  stop be so fucking gay you three people ain't nobody got that username username
label_nltk:  stop be so fucking gay you three ain't nobody got time for that username username username
